In [1]:
import os
import sys
import numpy as np
import pandas as pd
from glob import glob
os.chdir('C:/Users/Cole/Documents/GitHub/PLIC/Automation-Files/')
import Valid_Matched
import Scoring

# Setup necessary files and variables for later

In [2]:
os.chdir('C:/Users/Cole/Documents/PLIC_DATA/')
Weights = pd.read_excel('Weights_May2019.xlsx').transpose()[0]
Basedf = pd.read_csv('PLIC_May2019.csv', nrows = 1)
MainSurveys_Folder = 'SurveysMay2019/'
Questions = ['Q1b', 'Q1d', 'Q1e', 'Q2b', 'Q2d', 'Q2e', 'Q3b', 'Q3d', 'Q3e', 'Q4b']

# Create Valid/Matched Surveys Functions

In [3]:
def ConcatValidSurveys(FileList, ValidLocation, ValidFileName):
    if not os.path.exists(ValidLocation):
        os.mkdir(ValidLocation)

    dfs = []
    for f in FileList:
        Class_ID = 'R_' + f.split('_')[-2]
        df = pd.read_csv(f, skiprows = [1], dtype = {'Q5a':'object'})
        df['Class_ID'] = Class_ID
        df = df[(df['V5'] == 1) & (df['Unnamed: 7'] == 1) & (df['Q6d'] == 2)]
        df = df[(df['Qt1_3'] >= 30) | (df['Qt2_3'] >= 30) | (df['Qt3_3'] >= 30) | (df['Qt4_3'] >= 30)]
        df = df.drop_duplicates(subset = ['Q5b', 'Q5c'])
        if('Survey' not in df.columns):
            df['Survey'] = 'C'

        Items = [c for c in df.columns for Q in Questions if Q in c and 'TEXT' not in c and 'l' not in c and '_' in c]
        df[Items] = df[Items].fillna('0').astype(str).apply(lambda x: x.str.replace('^(?!0*$).*$', 
                                                                                    '1')).astype(float).replace(0, np.nan)

        df.to_csv(ValidLocation + f.replace('\\', '/').split('/')[-1].split('.')[0] + '_Valid.csv', index = False)
        dfs.append(df)
        
    df = pd.concat(dfs, axis = 0)
    df.to_csv('Collective_Surveys/' + ValidFileName, index = False)
    
    return(df)

def ConcatMatchedSurveys(PreFileList, PostFileList, PreMatchedLocation, PostMatchedLocation, PreCollectiveSurveyName, 
                         PostCollectiveSurveyName):
    if not os.path.exists(PreMatchedLocation):
        os.mkdir(PreMatchedLocation)
    if not os.path.exists(PostMatchedLocation):
        os.mkdir(PostMatchedLocation)
    
    Predfs = []
    Postdfs = []
    for f_pre in PreFileList:
        for f_post in PostFileList:
            if(f_pre.split('_')[-2] != f_post.split('_')[-2]):
                continue
            print(f_pre)
            Class_ID = 'R_' + f_pre.split('_')[-2] # Split of the underscore at the end for the new format
            # Change Valid_Matched to version of matching algorithm being used
            NPre, NPost, dfPre, dfPost = Valid_Matched.ValMat(PRE = pd.read_csv(f_pre, skiprows = [1], dtype = {'Q5a':'object'}), 
                                                              POST = pd.read_csv(f_post, skiprows = [1], dtype = {'Q5a':'object'}))
            if('Survey' not in dfPre.columns):
                dfPre['Survey'] = 'C'
            if('Survey' not in dfPost.columns):
                dfPost['Survey'] = 'C'
            dfPre['Class_ID'] = Class_ID
            dfPost['Class_ID'] = Class_ID
            Predfs.append(dfPre)
            Postdfs.append(dfPost)

            dfPre.to_csv(PreMatchedLocation + f_pre.replace('\\', '/').split('/')[-1].split('.')[0] + '_ValMat.csv', 
                           index = False)
            dfPost.to_csv(PostMatchedLocation + f_post.replace('\\', '/').split('/')[-1].split('.')[0] + '_ValMat.csv', 
                            index = False)
            
            break
    
    dfPre_Matched = pd.concat(Predfs, join = 'outer', axis = 0).reset_index(drop = True)
    dfPost_Matched = pd.concat(Postdfs, join = 'outer', axis = 0).reset_index(drop = True)

    dfPre_Matched.to_csv('Collective_Surveys/PRE_Valid_Matched/' + PreCollectiveSurveyName, index = False)
    dfPost_Matched.to_csv('Collective_Surveys/POST_Valid_Matched/' + PostCollectiveSurveyName, index = False)

    return(dfPre_Matched, dfPost_Matched)

def ConsentAtPost(PRE_Valid_File, PRE_Valid_Matched_File):
    Valid_df = pd.read_csv(PRE_Valid_File, dtype = {'Q5a':'object'})
    Matched_df = pd.read_csv(PRE_Valid_Matched_File, dtype = {'Q5a':'object'})

    # Add Pre-Surveys from matched set to overall set who enacted consent at POST
    ActualllyValidPre = Matched_df[~Matched_df['V1'].isin(Valid_df['V1'])]
    Total_Valid_df = pd.concat([Valid_df, ActualllyValidPre], join = 'inner', axis = 0)
    
    Total_Valid_df.to_csv(PRE_Valid_File, index = False)
    
    return(Total_Valid_df)

def MergeSurveys(PRE_Matched_File, POST_Matched_File, FileName):
    PRE_df = pd.read_csv(PRE_Matched_File, dtype = {'Q5a':'object'})
    POST_df = pd.read_csv(POST_Matched_File, dtype = {'Q5a':'object'})
    
    PRE_df_S = Scoring.CalcScore(PRE_df, Weights)
    POST_df_S = Scoring.CalcScore(POST_df, Weights)

    PRE_df_S = PRE_df_S.rename(columns = {'TotalScores':'PreScores'})
    POST_df_S = POST_df_S.rename(columns = {'TotalScores':'PostScores'})

    PRE_df_S['FullName'] = PRE_df_S['Q5b'].str.lower().str.replace(' ', '') + PRE_df_S['Q5c'].str.lower().str.replace(' ', '')
    POST_df_S['FullName'] = POST_df_S['Q5b'].str.lower().str.replace(' ', '') + POST_df_S['Q5c'].str.lower().str.replace(' ', '')
    POST_df_S['BackName'] = POST_df_S['Q5c'].str.lower().str.replace(' ', '') + POST_df_S['Q5b'].str.lower().str.replace(' ', '')
    
    Full_df = pd.merge(left = PRE_df_S, right = POST_df_S, how = 'inner', on = ['Class_ID', 'FullName'])
    Back_df = pd.merge(left = PRE_df_S, right = POST_df_S, how = 'inner', left_on = ['Class_ID', 'FullName'], 
                   right_on = ['Class_ID', 'BackName'])
    ID_df = pd.merge(left = PRE_df_S, right = POST_df_S, how = 'inner', on = ['Class_ID', 
                                                                              'Q5a']).rename(columns = {'Q5a':'Q5a_x'})
    ID_df['Q5a_y'] = ID_df['Q5a_x']
    
    Merged_df = pd.concat([Full_df, Back_df, ID_df], axis = 0, 
                          join = 'inner').drop_duplicates().drop(columns = ['BackName']).reset_index(drop = True)
    
    if('Q4b' in Merged_df.columns):
        Merged_df = Merged_df.rename(columns = {'Q1b':'Q1b_x', 'Q1d':'Q1d_x', 'Q1e':'Q1e_x', 'Q2b':'Q2b_x', 'Q2d':'Q2d_x',
                                                'Q2e':'Q2e_x', 'Q3b':'Q3b_x', 'Q3d':'Q3d_x', 'Q3e':'Q3e_x', 'Q4b':'Q4b_x'})
    
    Merged_df.to_csv('Collective_Surveys/Merged/' + FileName, index = False)
    
    return(Merged_df)

def MergePlusMissing(MergedFile, ValidPRE_File, ValidPOST_File, CompleteFileName):
    
    Merged_df = pd.read_csv('Collective_Surveys/Merged/' + MergedFile)

    PRE_df = pd.read_csv('Collective_Surveys/PRE_Valid/' + ValidPRE_File)
    POST_df = pd.read_csv('Collective_Surveys/POST_Valid/' + ValidPOST_File)

    PRE_df_S = Scoring.CalcScore(PRE_df, Weights).rename(columns = {'TotalScores':'PreScores'})
    POST_df_S = Scoring.CalcScore(POST_df, Weights).rename(columns = {'TotalScores':'PostScores'})

    Unmatched_PRE = PRE_df_S[~PRE_df_S['V1'].isin(Merged_df['V1_x'])]
    Unmatched_POST = POST_df_S[~POST_df_S['V1'].isin(Merged_df['V1_y'])]

    Unmatched_PRE.columns = [c + '_x' if c != 'Class_ID' and c != 'PreScores' else c for c in Unmatched_PRE.columns]
    Unmatched_POST.columns = [c + '_y' if c != 'Class_ID' and c != 'PostScores' else c for c in Unmatched_POST.columns]

    Complete_df = pd.concat([Merged_df, Unmatched_PRE, Unmatched_POST], axis = 0, join = 'outer')
    Complete_df = Complete_df[Merged_df.columns]
    Complete_df.to_csv('Collective_Surveys/Complete/' + CompleteFileName, index = False)

    return(Complete_df)

def ConcatSurveys(Semester, Year):
    PreFiles = glob(MainSurveys_Folder + Semester + Year + '/PRE/*.csv')
    PostFiles = glob(MainSurveys_Folder + Semester + Year + '/POST/*.csv')

    PREValid = ConcatValidSurveys(PreFiles, MainSurveys_Folder + Semester + Year + '/PRE/Valid/',
                                  'PRE_Valid/' + Semester + Year + '_PRE_Valid.csv')
    POSTValid = ConcatValidSurveys(PostFiles, MainSurveys_Folder + Semester + Year + '/POST/Valid/',
                                   'POST_Valid/' + Semester + Year + '_POST_Valid.csv')

    PREValMat, POSTValMat = ConcatMatchedSurveys(PreFiles, PostFiles,
                                                 MainSurveys_Folder + Semester + Year + '/PRE/Valid/Matched/',
                                                 MainSurveys_Folder + Semester + Year + '/POST/Valid/Matched/',
                                                 Semester + Year + '_PRE_ValMat.csv',
                                                 Semester + Year + '_POST_ValMat.csv')

    PREValid = ConsentAtPost('Collective_Surveys/PRE_Valid/' + Semester + Year + '_PRE_Valid.csv',
                             'Collective_Surveys/PRE_Valid_Matched/' + Semester + Year + '_PRE_ValMat.csv')
    
    return(PREValMat, POSTValMat)

def CompleteConcat(FolderName):
    Files = [f for f in glob('Collective_Surveys/' + FolderName + '/*') if 'Concat' not in f]
    
    print(Files)
    
    dfs = [pd.read_csv(f) for f in Files]
    df = pd.concat(dfs, join = 'outer', axis = 0)
    df.to_csv('Collective_Surveys/' + FolderName + '/' + FolderName + '_Concat.csv', index = False)
    
    return(df)

# Concatenate Fall 2017 Data

In [4]:
%%capture

Fall2017_PREValMat, Fall2017_POSTValMat = ConcatSurveys('Fall', '2017')

Fall2017_POSTValMat.loc[86, 'Q5c'] = 'Chris'
Fall2017_POSTValMat.loc[124, 'Q5c'] = 'Will'
Fall2017_PREValMat.loc[242, 'Q5c'] = 'Ray'
Fall2017_PREValMat.loc[270, 'Q5c'] = 'Chris'
Fall2017_PREValMat.loc[195, 'Q5c'] = 'Ben'
Fall2017_POSTValMat.loc[228, 'Q5c'] = 'Theo'
Fall2017_POSTValMat.loc[320, 'Q5c'] = 'Santi'
Fall2017_POSTValMat.loc[347, 'Q5c'] = 'Zac'
Fall2017_PREValMat.loc[417, 'Q5b'] = 'StLouis'
Fall2017_POSTValMat.loc[509, 'Q5c'] = 'Omar'
Fall2017_POSTValMat.loc[529, 'Q5c'] = 'Brad'

Fall2017_PREValMat.to_csv('Collective_Surveys/PRE_Valid_Matched/Fall2017_PRE_ValMat.csv', index = False)
Fall2017_POSTValMat.to_csv('Collective_Surveys/POST_Valid_Matched/Fall2017_POST_ValMat.csv', index = False)

Fall2017_Matched = MergeSurveys('Collective_Surveys/PRE_Valid_Matched/Fall2017_PRE_ValMat.csv',
                                'Collective_Surveys/POST_Valid_Matched/Fall2017_POST_ValMat.csv',
                                'Fall2017_Merged.csv')

Fall2017_Complete = MergePlusMissing('Fall2017_Merged.csv', 'Fall2017_PRE_Valid.csv', 'Fall2017_POST_Valid.csv', 
                                     'Fall2017_Complete.csv')

KeyError: 'Finished'

# Concatenate Spring 2018 Data

In [5]:
%%capture

Spring2018_PREValMat, Spring2018_POSTValMat = ConcatSurveys('Spring', '2018')

Spring2018_PREValMat.loc[8, 'Q5c'] = 'Joshua'
Spring2018_PREValMat.loc[12, 'Q5c'] = 'Daria'
Spring2018_POSTValMat.loc[5, 'Q5c'] = 'Greg'
Spring2018_POSTValMat.loc[148, 'Q5c'] = 'Edward'
Spring2018_PREValMat.loc[388, 'Q5c'] = 'Mourud'

Spring2018_PREValMat.to_csv('Collective_Surveys/PRE_Valid_Matched/Spring2018_PRE_ValMat.csv', index = False)
Spring2018_POSTValMat.to_csv('Collective_Surveys/POST_Valid_Matched/Spring2018_POST_ValMat.csv', index = False)

Spring2018_Matched = MergeSurveys('Collective_Surveys/PRE_Valid_Matched/Spring2018_PRE_ValMat.csv', 
                                  'Collective_Surveys/POST_Valid_Matched/Spring2018_POST_ValMat.csv',
                                  'Spring2018_Merged.csv')

Spring2018_Complete = MergePlusMissing('Spring2018_Merged.csv', 'Spring2018_PRE_Valid.csv', 'Spring2018_POST_Valid.csv',
                                       'Spring2018_Complete.csv')

# Concatenate Fall 2018 Data

In [6]:
%%capture

Fall2018_PREValMat, Fall2018_POSTValMat = ConcatSurveys('Fall', '2018')

Fall2018_PREValMat.loc[8, 'Q5c'] = 'Alex'
Fall2018_POSTValMat.loc[50, 'Q5c'] = 'Alex'
Fall2018_PREValMat.loc[280, 'Q5a'] = '900725834'
Fall2018_PREValMat.loc[380, 'Q5c'] = 'Chris'
Fall2018_POSTValMat.loc[581, 'Q5c'] = 'Christopher'
Fall2018_PREValMat.loc[606, 'Q5c'] = 'Cece'
Fall2018_PREValMat.loc[716, 'Q5c'] = 'Ben'
Fall2018_POSTValMat.loc[737, 'Q5c'] = 'Sam'
Fall2018_POSTValMat.loc[730, 'Q5c'] = 'Rob'
Fall2018_PREValMat.loc[1042, 'Q5c'] = 'Kamsi'
Fall2018_POSTValMat.loc[1028, 'Q5c'] = 'Alex'
Fall2018_POSTValMat.loc[1054, 'Q5c'] = 'Tiffany'
Fall2018_POSTValMat.loc[1167, 'Q5c'] = 'Alex'

Fall2018_PREValMat.to_csv('Collective_Surveys/PRE_Valid_Matched/Fall2018_PRE_ValMat.csv', index = False)
Fall2018_POSTValMat.to_csv('Collective_Surveys/POST_Valid_Matched/Fall2018_POST_ValMat.csv', index = False)

Fall2018_Matched = MergeSurveys('Collective_Surveys/PRE_Valid_Matched/Fall2018_PRE_ValMat.csv', 
                                'Collective_Surveys/POST_Valid_Matched/Fall2018_POST_ValMat.csv',
                                'Fall2018_Merged.csv')

Fall2018_Complete = MergePlusMissing('Fall2018_Merged.csv', 'Fall2018_PRE_Valid.csv', 'Fall2018_POST_Valid.csv',
                                     'Fall2018_Complete.csv')

Out_Predf = Fall2018_PREValMat[~Fall2018_PREValMat['V1'].isin(Fall2018_Matched['V1_x'])]
Out_Postdf = Fall2018_POSTValMat[~Fall2018_POSTValMat['V1'].isin(Fall2018_Matched['V1_y'])]

print(Out_Predf[['Q5a', 'Q5b', 'Q5c']])
print(Out_Postdf[['Q5a', 'Q5b', 'Q5c']])

# Concatenate Spring 2019 Data

In [7]:
%%capture

Spring2019_PREValMat, Spring2019_POSTValMat = ConcatSurveys('Spring', '2019')

Spring2019_POSTValMat.loc[27, 'Q5b'] = 'Rocha'
Spring2019_POSTValMat.loc[115, 'Q5c'] = 'Alex'
Spring2019_PREValMat.loc[131, 'Q5c'] = 'Zach'
Spring2019_PREValMat.loc[194, 'Q5c'] = 'Josh' 
Spring2019_PREValMat.loc[651, 'Q5c'] = 'Tim'
Spring2019_POSTValMat.loc[705, 'Q5c'] = 'Josh'

Spring2019_PREValMat.to_csv('Collective_Surveys/PRE_Valid_Matched/Spring2019_PRE_ValMat.csv', index = False)
Spring2019_POSTValMat.to_csv('Collective_Surveys/POST_Valid_Matched/Spring2019_POST_ValMat.csv', index = False)

Spring2019_Matched = MergeSurveys('Collective_Surveys/PRE_Valid_Matched/Spring2019_PRE_ValMat.csv', 
                                'Collective_Surveys/POST_Valid_Matched/Spring2019_POST_ValMat.csv',
                                'Spring2019_Merged.csv')

Out_Predf = Spring2019_PREValMat[~Spring2019_PREValMat['V1'].isin(Spring2019_Matched['V1_x'])]
Out_Postdf = Spring2019_POSTValMat[~Spring2019_POSTValMat['V1'].isin(Spring2019_Matched['V1_y'])]

print(Out_Predf[['Q5a', 'Q5b', 'Q5c']])
print(Out_Postdf[['Q5a', 'Q5b', 'Q5c']])

Spring2019_Complete = MergePlusMissing('Spring2019_Merged.csv', 'Spring2019_PRE_Valid.csv', 'Spring2019_POST_Valid.csv',
                                       'Spring2019_Complete.csv')

# Concatenate Fall 2019 Data

In [5]:
%%capture

Fall2019_PREValMat, Fall2019_POSTValMat = ConcatSurveys('Fall', '2019')

Fall2019_PREValMat.loc[7, 'Q5c'] = 'Tamaria '
Fall2019_POSTValMat.loc[202, 'Q5c'] = 'Connie'
Fall2019_PREValMat.loc[534, 'Q5c'] = 'Benjamin'
Fall2019_PREValMat.loc[711, 'Q5c'] = 'Alex'
Fall2019_POSTValMat.loc[1082, 'Q5c'] = 'Chris'
Fall2019_POSTValMat.loc[1095, 'Q5c'] = 'Aidi'
Fall2019_PREValMat.loc[1336, 'Q5c'] = 'Ziyu'
Fall2019_POSTValMat.loc[1203, 'Q5c'] = 'Jichun'
Fall2019_PREValMat.loc[1260, 'Q5c'] = 'jeff'
Fall2019_POSTValMat.loc[1225, 'Q5c'] = 'Ben'
Fall2019_PREValMat.loc[1657, 'Q5c'] = 'Jonathan'
Fall2019_PREValMat.loc[1992, 'Q5c'] = 'Parker'

Fall2019_PREValMat.to_csv('Collective_Surveys/PRE_Valid_Matched/Fall2019_PRE_ValMat.csv', index = False)
Fall2019_POSTValMat.to_csv('Collective_Surveys/POST_Valid_Matched/Fall2019_POST_ValMat.csv', index = False)

Fall2019_Matched = MergeSurveys('Collective_Surveys/PRE_Valid_Matched/Fall2019_PRE_ValMat.csv',
                                'Collective_Surveys/POST_Valid_Matched/Fall2019_POST_ValMat.csv',
                                'Fall2019_Merged.csv')

Out_Predf = Fall2019_PREValMat[~Fall2019_PREValMat['V1'].isin(Fall2019_Matched['V1_x'])]
Out_Postdf = Fall2019_POSTValMat[~Fall2019_POSTValMat['V1'].isin(Fall2019_Matched['V1_y'])]

print(Out_Predf[['Q5a', 'Q5b', 'Q5c']])
print(Out_Postdf[['Q5a', 'Q5b', 'Q5c']])

Fall2019_Complete = MergePlusMissing('Fall2019_Merged.csv', 'Fall2019_PRE_Valid.csv', 'Fall2019_POST_Valid.csv',
                                     'Fall2019_Complete.csv')

SurveysMay2019/Fall2019/PRE\Fall2019_Agnes_Scott_College_202_Marine_PRE_R_2Cmq05OZWF8UESV_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Barnard_College_PHYS2001_Savin_PRE_R_2Cx014FHcWCOw9u_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Bethel_University_PHY_292D_Hogan_PRE_R_XH3AUfCAn0ph6Hn_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Cornell_University_Phys6510_Holmes_PRE_R_2XhpZkn4DGP598r_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Cornell_University_Phys_1112_Holmes_PRE_R_9EVBSZgwQyP6mWZ_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Cornell_University_Phys_2213_Holmes_PRE_R_31hmnBMkK3Bvc3z_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Cornell_University_Phys_2218_Holmes_PRE_R_1pyQ0Ym19YJPe3Q_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Penn_State_University_PHYS_457_Purdy_Drew_PRE_R_3q88ROCl8ybpqTf_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Salem_State_University_PHS_311_Conlin_PRE_R_2cC5NePDupfT60Q_May2019.csv
SurveysMay2019/Fall2019/PRE\Fall2019_Siena_

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (33,46,75,82,95,128,142,172,192,204) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:130: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Concatenate all surveys

In [11]:
PRE_Valid_df = CompleteConcat('PRE_Valid')
POST_Valid_df = CompleteConcat('POST_Valid')
PRE_ValMat_df = CompleteConcat('PRE_Valid_Matched')
POST_ValMat_df = CompleteConcat('POST_Valid_Matched')
Merged_df = CompleteConcat('Merged')
Complete_df = CompleteConcat('Complete')

['Collective_Surveys/PRE_Valid\\Fall2017_PRE_Valid.csv', 'Collective_Surveys/PRE_Valid\\Fall2018_PRE_Valid.csv', 'Collective_Surveys/PRE_Valid\\Fall2019_PRE_Valid.csv', 'Collective_Surveys/PRE_Valid\\Spring2018_PRE_Valid.csv', 'Collective_Surveys/PRE_Valid\\Spring2019_PRE_Valid.csv']


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



['Collective_Surveys/POST_Valid\\Fall2017_POST_Valid.csv', 'Collective_Surveys/POST_Valid\\Fall2018_POST_Valid.csv', 'Collective_Surveys/POST_Valid\\Fall2019_POST_Valid.csv', 'Collective_Surveys/POST_Valid\\Spring2018_POST_Valid.csv', 'Collective_Surveys/POST_Valid\\Spring2019_POST_Valid.csv']


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (214) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['Collective_Surveys/PRE_Valid_Matched\\Fall2017_PRE_ValMat.csv', 'Collective_Surveys/PRE_Valid_Matched\\Fall2018_PRE_ValMat.csv', 'Collective_Surveys/PRE_Valid_Matched\\Fall2019_PRE_ValMat.csv', 'Collective_Surveys/PRE_Valid_Matched\\Spring2018_PRE_ValMat.csv', 'Collective_Surveys/PRE_Valid_Matched\\Spring2019_PRE_ValMat.csv']
['Collective_Surveys/POST_Valid_Matched\\Fall2017_POST_ValMat.csv', 'Collective_Surveys/POST_Valid_Matched\\Fall2018_POST_ValMat.csv', 'Collective_Surveys/POST_Valid_Matched\\Fall2019_POST_ValMat.csv', 'Collective_Surveys/POST_Valid_Matched\\Spring2018_POST_ValMat.csv', 'Collective_Surveys/POST_Valid_Matched\\Spring2019_POST_ValMat.csv']
['Collective_Surveys/Merged\\Fall2017_Merged.csv', 'Collective_Surveys/Merged\\Fall2018_Merged.csv', 'Collective_Surveys/Merged\\Fall2019_Merged.csv', 'Collective_Surveys/Merged\\Spring2018_Merged.csv', 'Collective_Surveys/Merged\\Spring2019_Merged.csv']


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (24,33,46,75,82,121,142,172,291,300,313,342,349,362,388,409,439,471) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (33,46,75,82,95,128,142,172,192,204) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['Collective_Surveys/Complete\\Fall2017_Complete.csv', 'Collective_Surveys/Complete\\Fall2018_Complete.csv', 'Collective_Surveys/Complete\\Fall2019_Complete.csv', 'Collective_Surveys/Complete\\Spring2018_Complete.csv', 'Collective_Surveys/Complete\\Spring2019_Complete.csv']


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,24,33,46,59,75,82,95,109,121,128,139,142,161,172,182,187,188,189,192,195,204,220,254,291,300,313,342,349,362,388,395,409,439,459,471,485) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (33,142,313,342,349,362) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (4,7,15,19,30,33,47,50,63,66,78,81,96,99,102,113,117,126,145,147,150,153,165,166,167,168,171,174,183,199,258,262,263,264,265,311,324,360,373,399,406,465,470,482) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Anaconda3